In [6]:
%pip install -q \
    fuzzywuzzy \
    easyocr \
    pandas \
    requests

Note: you may need to restart the kernel to use updated packages.


In [28]:
# seed reference items using api

# import from csv
import pandas as pd
import requests

referenceitems_df = pd.read_csv('referenceitems.csv')

def writeReferenceItem(json):
    url = 'http://localhost:3001/api/referenceItem'
    response = requests.post(url, json=json)
    if response.status_code == 200:
        print(f'Reference item successfully written. Response:\n{response.text}')
    else:
        print(f'Failed to write reference item. Response:\n{response.text}')
    return response

for index, row in referenceitems_df.iterrows():
    response = writeReferenceItem({
        "name": row['name'],
        "quantity": row['quantity'],
        "unitOfMeasure": row['unitofmeasure'],
        "price": row['price'],
        "pricePerWeight": row['priceperweight'],
        "referenceUrl": row['referenceurl']
    })
    new_referenceitem_id = response.json()['id']
    referenceitems_df.at[index, 'id'] = int(new_referenceitem_id)

# test
# writeReferenceItem({
#     "name": "Sample item",
#     "quantity": 10,
#     "unitOfMeasure": "g",
#     "price": 100,
#     "pricePerWeight": 10,
#     "referenceUrl": "test.com"
# })

Reference item successfully written. Response:
{"id":50,"name":"BBQ Sauce","quantity":455,"unitOfMeasure":"g","price":2.47,"pricePerWeight":0.005429,"referenceUrl":"https://walmart.ca/#missing","createdAt":"2024-07-31T02:31:17.221Z","updatedAt":"2024-07-31T02:31:17.221Z"}
Reference item successfully written. Response:
{"id":51,"name":"Beans","quantity":540,"unitOfMeasure":"g","price":1.37,"pricePerWeight":0.002537,"referenceUrl":"https://walmart.ca/#missing","createdAt":"2024-07-31T02:31:17.231Z","updatedAt":"2024-07-31T02:31:17.231Z"}
Reference item successfully written. Response:
{"id":52,"name":"Bouillon cubes","quantity":80,"unitOfMeasure":"g","price":1.14,"pricePerWeight":0.01425,"referenceUrl":"https://walmart.ca/#missing","createdAt":"2024-07-31T02:31:17.240Z","updatedAt":"2024-07-31T02:31:17.240Z"}
Reference item successfully written. Response:
{"id":53,"name":"Candy","quantity":150,"unitOfMeasure":"g","price":1,"pricePerWeight":0.006667,"referenceUrl":"https://walmart.ca/#miss

In [29]:
referenceitems_df

,name,quantity,unitofmeasure,price,priceperweight,referenceurl,id
0,BBQ Sauce,455,g,2.47,0.005429,https://walmart.ca/#missing,50.0
1,Beans,540,g,1.37,0.002537,https://walmart.ca/#missing,51.0
2,Bouillon cubes,80,g,1.14,0.014250,https://walmart.ca/#missing,52.0
3,Candy,150,g,1.00,0.006667,https://walmart.ca/#missing,53.0
4,Cereal,755,g,4.97,0.006583,https://walmart.ca/#missing,54.0
5,Chia seeds,454,g,5.47,0.012048,https://walmart.ca/#missing,55.0
6,Chips,280,g,2.18,0.007786,https://walmart.ca/#missing,56.0
7,Chocolate,100,g,0.60,0.006000,https://walmart.ca/#missing,57.0
8,Chocolate chips,300,g,3.97,0.013233,https://walmart.ca/#missing,58.0
9,Cookies,70,g,0.67,0.009571,https://walmart.ca/#missing,59.0


In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

# third attempt
import easyocr
import pandas as pd

# - "upload" images (go through folder) -- i'll take it out for now

# chron/service-ish
def cropImages(rawFiles):
    # ... do the things
    # store ids of processed files
    # return file ids
    pass

# data API
def get_image_url(croppedImageID):
    # Q: not sure if this is a good design to pass ID and fetch URL every time, current rationale is the async nature of chron/service-ish approach
    # ✨ this is the stuff that I'd love to not re-parse every time in my notebook and have to wait!
    # TODO: implement
    # get image url by id
    return './receipts/IMG_4553.jpg' # for now

def writeReceiptTexts(json_data):
    print(f'Would need to write receipt texts: {json_data}')
    # for now just return an array with the IDs from json_data
    return [123, 124]


reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory

def ocrImage(croppedImageID):
    image_url = get_image_url(croppedImageID)
    receipt_strings = reader.readtext(image_url)

    # guard clause
    if len(receipt_strings) < 1:
        return # no text found
    
    # ... do the things

    # "receiptTexts": [
    #                     {
    #                         "id": "123",
    #                         "text": "Item 1",
    #                         "boundingBox": "{'x':0,'y':0,'width':100,'height':20}", 
    #                         "imageFileId": "123"
    #                     },
    #                     {
    #                         "id": "124",
    #                         "text": "Item 2",
    #                         "boundingBox": "{'x':0,'y':0,'width':100,'height':20}", 
    #                         "imageFileId": "123"
    #                     }
    #                 ]

    # transformation
    receipt_strings_df = pd.DataFrame(receipt_strings, columns=['boundingBox', 'text', 'confidence'])
    receipt_strings_df.drop(columns=['confidence'], inplace=True)
    receipt_strings_df['imageFileId'] = croppedImageID

    # push to API
    json_data = receipt_strings_df.to_json(orient='records')
    response = writeReceiptTexts(json_data)

    return response # return list of IDs, but do chron job later if not used (best or both)

def crop_images(images):
    return 123 # assume return ids
    # { # static for now, TODO: implement
    #     "data": [
    #         {
    #             "id": "123",
    #             "filename": "IMG_5903.jpg"
    #         }
    #     ]
    # }

choices = { # static for now, TODO: pull from csv, but also need to pull from API to get proper reference item IDs
    "data": [
        {
            "id": "123",
            "query": "Schar Gluten Free Hamburger Buns",
            "referenceItemId": "123"
        },
        {
            "id": "124",
            "query": "Promise Brioche Buns",
            "referenceItemId": "123"
        }
    ]
}
choices_dict = {val['id']: val['query'] for val in choices['data']}
def get_reference_item_id(best_match):
    for choice in choices:
        if choice['query'] == best_match:
            return choice['referenceItemId']

# consider global API function: query or mutation
# async def api_call(queryormutation, variable_values):
    # try:
    #     data = await client.execute_async(queryormutation, variable_values=variable_values)
    #     print(data)
    # except Exception as e:
    #     print(f"Query failed: {e}")

def fetch_receipt(receiptID):
# TODO: implement
# async def fetch_receipt_details():
    # try:
    #     data = await client.execute_async(query, variable_values={"receiptID": receiptID})
    #     print(data)
    # except Exception as e:
    #     print(f"Query failed: {e}")
    return {
        "data": {
            "receipt": {
                # "vendor": {
                #     "id": "123",
                #     "name": "Vendor Name"
                # },
                # "images": [
                #     {
                #         "id": "123",
                #         "filename": "IMG_5903.jpg"
                #     }
                # ],
                # "receiptTexts": [
                #                     {
                #                         "id": "123",
                #                         "text": "Item 1",
                #                         "boundingBox": "{'x':0,'y':0,'width':100,'height':20}", 
                #                         "imageFileId": "123"
                #                     },
                #                     {
                #                         "id": "124",
                #                         "text": "Item 2",
                #                         "boundingBox": "{'x':0,'y':0,'width':100,'height':20}", 
                #                         "imageFileId": "123"
                #                     }
                #                 ]
            }
        }
    }

def writeEligibleExpense(variable_values):
# TODO: implement
# async def write_eligible_expense():
    # try:
        # data = await client.execute_async(mutation, variable_values=variable_values)
        # print(data)
    # except Exception as e:
        # print(f"Mutation failed: {e}")
    pass

def parseTextForEligibleExpenses(receiptID):
    receipt = fetch_receipt(receiptID)
    receipt_texts = receipt['data']['receipt']['receiptTexts']
    threshold = 80

    # TODO: some clean up before look up
        # trim numbers in the beginning of the string, drop too short alphanumeric strings
            # search_query = re.sub(r'^\d{1,10}', '', search_query)
            # # if len(search_query) > 2:
        # if search_query contains at least one digit or letter
            # if any(char.isalnum() for char in search_query):
        # regex out 1 to 10 digits in the beginning of the string for sku codes in store receipts
    
    # for now choices are static
    # choices = get_choices() # receipt_text, receipt_vendor)
    # TODO: clean up choices too 
        # avoid strict substring matches
            # len(search_query)/len(best_match) < 0.4
        # here I could also optimize for a given vendor if available
            # receipt_vendor = receipt['data']['receipt']['vendor']['name']

    for receipt_text in receipt_texts:
        receipt_text_id = receipt_text['id']
        receipt_text = receipt_text['text']
        _, best_score, best_match_id = process.extractOne(receipt_text, choices_dict, scorer=fuzz.token_set_ratio)
        if best_score >= threshold:
            # found a decent match
            # get corresponding referenceItem 
            reference_item_id = choices[best_match_id]['referenceItemId']
            # create eligible expense
            writeEligibleExpense(
                {
                    'receiptTextId': receipt_text_id,
                    'referenceItemId': reference_item_id,
                    'quantiy': 1, # TODO: find quantity
                    'priceEach': 0 # TODO: find price
                }
            )

def main_single_receipt(images): # can test with one image, but could be multiple images per receipt (or PDF file, etc)
    # direct
    # new_images = upload_images(images) # assume return ids

    # "chron/service-style", UI says processing in background
    # cropped_images = crop_images(new_images) # assume return ids
    # skip for now
    cropped_images = [123]
    receiptTexts = ocrImage(cropped_images[0]) # TODO: for loop images
    # ... need to match with reference items
    # createReceipt(receiptTexts)
    return 123 # receiptID

main_single_receipt('_')
# ./store/optimized_images/1.jpg ... url idea

# def main_bulk(folder):
#     # images = os.listdir(folder)
#     # for image in images:
#     #     main_single_receipt(image) # maybe async
#     pass

Would need to write receipt texts: [{"boundingBox":[[141,101],[619,101],[619,205],[141,205]],"text":"IFRESHCO","imageFileId":123},{"boundingBox":[[87,232],[594,232],[594,280],[87,280]],"text":"CARLETON PLACE LANSDOWNE FRESHCO _","imageFileId":123},{"boundingBox":[[179,270],[499,270],[499,310],[179,310]],"text":"110 Lansdowne Avenue","imageFileId":123},{"boundingBox":[[103,304],[237,304],[237,340],[103,340]],"text":"Car Ieton","imageFileId":123},{"boundingBox":[[245,309],[265,309],[265,335],[245,335]],"text":"P","imageFileId":123},{"boundingBox":[[273,301],[377,301],[377,337],[273,337]],"text":"ace ,ON","imageFileId":123},{"boundingBox":[[395,295],[587,295],[587,334],[395,334]],"text":"613-253-6141","imageFileId":123},{"boundingBox":[[178,322],[513,322],[513,372],[178,372]],"text":"HST# 844058206 RTOOO1","imageFileId":123},{"boundingBox":[[45,398],[203,398],[203,435],[45,435]],"text":"Served bv:","imageFileId":123},{"boundingBox":[[217,399],[269,399],[269,431],[217,431]],"text":"D M","i

123